In [1]:
import pandas as pd
import plotly.express as px
import os 


In [2]:
test_parquet=os.path.join(os.getcwd(), 'data', 'asl-signs', 'train_landmark_files','37055', "1003109377.parquet") #this is on the dataset
df= pd.read_parquet(test_parquet) #this is on the dataset

In [5]:
df.head()

,frame,row_id,type,landmark_index,x,y,z
0,14,14-face-0,face,0,0.432380,0.375417,-0.044000
1,14,14-face-1,face,1,0.437980,0.349091,-0.070176
2,14,14-face-2,face,2,0.429624,0.360922,-0.040331
3,14,14-face-3,face,3,0.430260,0.315678,-0.049688
4,14,14-face-4,face,4,0.441155,0.338613,-0.073218


In [6]:
df.columns

Index(['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z'], dtype='object')

In [7]:
df

,frame,row_id,type,landmark_index,x,y,z
0,14,14-face-0,face,0,0.432380,0.375417,-0.044000
1,14,14-face-1,face,1,0.437980,0.349091,-0.070176
2,14,14-face-2,face,2,0.429624,0.360922,-0.040331
3,14,14-face-3,face,3,0.430260,0.315678,-0.049688
4,14,14-face-4,face,4,0.441155,0.338613,-0.073218
...,...,...,...,...,...,...,...
12484,36,36-right_hand-16,right_hand,16,NaN,NaN,NaN
12485,36,36-right_hand-17,right_hand,17,NaN,NaN,NaN
12486,36,36-right_hand-18,right_hand,18,NaN,NaN,NaN
12487,36,36-right_hand-19,right_hand,19,NaN,NaN,NaN


In [3]:
px.scatter_3d(df, x="x", y="y", z="z", color="type" , opacity=0.5, width=800, height=800).show() #dataset

In [4]:
df2=pd.read_parquet("2d_landmarks.parquet") #rgb image landmarks
df2.columns

Index(['type', 'landmark_index', 'x', 'y', 'z', 'frame'], dtype='object')

In [23]:
df2

,type,landmark_index,x,y,z,frame
0,face,0,0.401075,0.314018,-0.013286,1
1,face,1,0.396080,0.286855,-0.021268,1
2,face,2,0.400056,0.295254,-0.011867,1
3,face,3,0.393460,0.264410,-0.013408,1
4,face,4,0.395249,0.279382,-0.022355,1
...,...,...,...,...,...,...
3796,right_hand,16,0.125883,0.907843,-0.038000,7
3797,right_hand,17,0.183508,0.835019,-0.026828,7
3798,right_hand,18,0.171646,0.864476,-0.036678,7
3799,right_hand,19,0.166592,0.891244,-0.041308,7


In [5]:
df2['z'].unique()

array([-0.01223104, -0.02338137, -0.01194532, ..., -0.03294123,
       -0.04047822, -0.04498838])

In [6]:
px.scatter_3d(df2, x="x", y="y", z="z", color="type" , opacity=0.5, width=800, height=800).show() #rgb image landmarks

In [11]:
df3= pd.read_parquet("3d_landmarks.parquet") #rgb image landmarks

In [12]:
df3

,frame,type,landmark_index,x,y,z
0,1,face,0,NaN,NaN,NaN
1,1,face,1,NaN,NaN,NaN
2,1,face,2,NaN,NaN,NaN
3,1,face,3,NaN,NaN,NaN
4,1,face,4,NaN,NaN,NaN
...,...,...,...,...,...,...
10312,19,right_hand,16,0.040310,0.862916,1.019
10313,19,right_hand,17,0.115405,0.794447,1.902
10314,19,right_hand,18,0.091971,0.811663,1.741
10315,19,right_hand,19,0.078073,0.837304,1.538


In [13]:
df3.columns

Index(['frame', 'type', 'landmark_index', 'x', 'y', 'z'], dtype='object')

In [14]:
px.scatter_3d(df3, x="x", y="y", z="z", color="type" , opacity=0.5, width=800, height=800).show()

In [17]:
print(df.shape, df2.shape, df3.shape)

(12489, 7) (10317, 6) (739, 6)
